In [195]:
#import packages
import pandas as pd
from prophet import Prophet
import warnings
warnings.filterwarnings('ignore')

In [196]:
# Load datasets
ingredients_df = pd.read_csv("Cleaned_datasets/ingredients_data_cleaned.csv")
sales_df = pd.read_csv("Cleaned_datasets/sales_data_cleaned.csv")

In [197]:
# convert data column to datetime dtype
sales_df['order_date'] = pd.to_datetime(sales_df['order_date'])
# extract needed columns from sales_df
pizza_df = sales_df[['order_date', 'pizza_name', 'quantity']]
# Group by order date and sum the quantity of each pizza sold
pizza_df = pizza_df.groupby(['order_date', 'pizza_name']).sum().reset_index()
pizza_df.head()

,order_date,pizza_name,quantity
0,2015-01-01,the barbecue chicken pizza,11
1,2015-01-01,the big meat pizza,5
2,2015-01-01,the calabrese pizza,1
3,2015-01-01,the california chicken pizza,4
4,2015-01-01,the chicken alfredo pizza,2


In [198]:
# transform data for time series modeling
sales_pivot = pizza_df.pivot(index='order_date', columns='pizza_name', values='quantity').fillna(0)
sales_pivot

pizza_name,the barbecue chicken pizza,the big meat pizza,the brie carre pizza,the calabrese pizza,the california chicken pizza,the chicken alfredo pizza,the chicken pesto pizza,the classic deluxe pizza,the five cheese pizza,the four cheese pizza,...,the prosciutto and arugula pizza,the sicilian pizza,the soppressata pizza,the southwest chicken pizza,the spicy italian pizza,the spinach and feta pizza,the spinach pesto pizza,the spinach supreme pizza,the thai chicken pizza,the vegetables + vegetables pizza
order_date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,11.0,5.0,0.0,1.0,4.0,2.0,4.0,10.0,6.0,5.0,...,4.0,4.0,0.0,7.0,5.0,2.0,2.0,5.0,7.0,3.0
2015-01-02,12.0,5.0,3.0,7.0,15.0,3.0,2.0,13.0,5.0,6.0,...,6.0,5.0,3.0,9.0,9.0,4.0,5.0,3.0,8.0,1.0
2015-01-03,2.0,3.0,1.0,1.0,6.0,0.0,1.0,4.0,5.0,4.0,...,1.0,2.0,0.0,9.0,4.0,5.0,3.0,0.0,4.0,6.0
2015-01-04,2.0,3.0,0.0,2.0,3.0,3.0,2.0,8.0,7.0,6.0,...,4.0,8.0,6.0,10.0,4.0,1.0,4.0,6.0,3.0,5.0
2015-01-05,6.0,1.0,1.0,4.0,11.0,2.0,3.0,9.0,4.0,4.0,...,5.0,5.0,0.0,6.0,5.0,5.0,2.0,4.0,10.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-27,4.0,3.0,1.0,1.0,3.0,0.0,3.0,8.0,2.0,2.0,...,1.0,4.0,1.0,2.0,4.0,1.0,3.0,1.0,5.0,9.0
2015-12-28,2.0,3.0,1.0,1.0,5.0,2.0,3.0,9.0,2.0,4.0,...,5.0,4.0,5.0,3.0,2.0,3.0,2.0,2.0,6.0,3.0
2015-12-29,4.0,2.0,1.0,2.0,4.0,1.0,1.0,3.0,3.0,3.0,...,3.0,2.0,3.0,1.0,3.0,1.0,4.0,0.0,5.0,1.0


FORECAST NEXT 7 DAYS

In [199]:
# Dictionary to store predictions
predictions_prophet = {}

# Loop through each pizza type and train Prophet model
for pizza_name in sales_pivot.columns:
    try:
        # Prepare data for Prophet
        df = sales_pivot[[pizza_name]].reset_index()
        df.columns = ['ds', 'y']  # Prophet requires columns named 'ds' and 'y'

        # Fit the model
        model = Prophet(weekly_seasonality=True)
        model.fit(df)

        # Create future dates (next 7 days)
        future = model.make_future_dataframe(periods=7, freq='D')

        # Generate forecast
        forecast = model.predict(future)

        # Store only relevant predictions
        predictions_prophet[pizza_name] = forecast[['ds', 'yhat']].set_index('ds').rename(columns={'yhat': pizza_name})

    except Exception as e:
        print(f"Skipping {pizza_name} due to an error: {e}")

# Combine all predictions into a single DataFrame
predictions_df = pd.concat(predictions_prophet, axis=1)
predictions_df.columns = predictions_df.columns.droplevel(0)  # Drop the top level of the column index
predictions_df = predictions_df.tail(7)
predictions_df

,the barbecue chicken pizza,the big meat pizza,the brie carre pizza,the calabrese pizza,the california chicken pizza,the chicken alfredo pizza,the chicken pesto pizza,the classic deluxe pizza,the five cheese pizza,the four cheese pizza,...,the prosciutto and arugula pizza,the sicilian pizza,the soppressata pizza,the southwest chicken pizza,the spicy italian pizza,the spinach and feta pizza,the spinach pesto pizza,the spinach supreme pizza,the thai chicken pizza,the vegetables + vegetables pizza
ds,,,,,,,,,,,,,,,,,,,,,
2016-01-01,7.037887,4.951689,1.484724,2.830294,6.831083,3.357052,3.193108,7.653488,3.986496,5.642348,...,4.644131,5.994641,2.857282,5.799839,5.562450,4.571400,3.406533,2.942635,6.730517,4.380344
2016-01-02,6.299159,5.029011,1.217387,2.839404,5.812976,2.940253,2.469717,7.072362,2.943251,5.579987,...,3.735435,5.996070,2.553831,4.977142,5.325831,3.470703,2.739437,2.346077,6.413686,4.366876
2016-01-03,4.700721,4.364114,1.511222,2.281558,5.035795,2.155344,2.500520,5.772282,2.775812,4.391448,...,3.612774,4.347425,2.135823,4.232634,4.235159,3.366429,2.497879,2.006320,5.481311,3.768260
2016-01-04,5.347371,4.651227,1.443492,3.089176,5.952823,2.757588,2.771182,6.811251,3.027536,4.721162,...,3.664551,4.354531,2.137526,5.258796,4.718965,3.952011,2.448607,2.183903,6.850571,3.962795
2016-01-05,5.724242,4.316668,1.583450,2.743206,5.332053,2.498318,2.004957,6.938037,3.580693,4.733678,...,4.216435,4.784415,3.034418,5.073097,5.033692,3.355198,2.720300,2.211541,6.144613,3.542593
2016-01-06,5.398943,4.605382,1.602660,2.897066,5.639627,2.536871,2.762721,6.591040,3.118050,5.271875,...,3.754961,5.399885,2.572811,4.764955,4.994057,3.835985,2.432316,2.250036,5.971623,4.293518
2016-01-07,6.447150,5.008372,1.737835,2.841662,5.995109,2.742665,2.953441,6.939367,3.522507,5.732380,...,3.749406,5.369175,2.601518,5.406941,5.584862,4.005361,2.637955,2.464121,6.519852,4.510614


In [200]:
ingredients_df = ingredients_df[['pizza_name', 'pizza_ingredients', 'Items_Qty_In_Grams']]
ingredients_df.rename(columns={'Items_Qty_In_Grams': 'items_qty'}, inplace=True)
ingredients_df.head()

,pizza_name,pizza_ingredients,items_qty
0,The Barbecue Chicken Pizza,Barbecued Chicken,40.0
1,The Barbecue Chicken Pizza,Red Peppers,15.0
2,The Barbecue Chicken Pizza,Green Peppers,20.0
3,The Barbecue Chicken Pizza,Tomatoes,30.0
4,The Barbecue Chicken Pizza,Red Onions,60.0


In [201]:
# Create a dictionary to store the ingredient quantities
ingredient_quantities = {}

# Ensure `pizza_name` values are correctly formatted
ingredients_df['pizza_name'] = ingredients_df['pizza_name'].str.strip().str.lower()
predictions_df.columns = predictions_df.columns.str.strip().str.lower()

# Iterate through each pizza in the predictions
for pizza_name in predictions_df.columns:
    if pizza_name in ingredients_df['pizza_name'].values:  # Ensure pizza exists in ingredients_df
        # Get the predicted quantity for the pizza
        predicted_quantity = predictions_df[pizza_name].sum()

        # Get the ingredients for the pizza
        pizza_ingredients = ingredients_df[ingredients_df['pizza_name'] == pizza_name]

        # Iterate through each ingredient for the pizza
        for _, row in pizza_ingredients.iterrows():
            ingredient = row['pizza_ingredients']
            ingredient_qty = row['items_qty']

            # Calculate the required quantity of the ingredient
            required_quantity = predicted_quantity * ingredient_qty

            # Add the required quantity to the dictionary
            if ingredient not in ingredient_quantities:
                ingredient_quantities[ingredient] = 0
            ingredient_quantities[ingredient] += required_quantity
    else:
        print(f"Warning: No ingredients found for pizza '{pizza_name}'")

# Create a DataFrame from the ingredient quantities
ingredient_requirements_df = pd.DataFrame.from_dict(
    ingredient_quantities, orient='index', columns=['required_quantity']
)

In [202]:
ingredient_requirements_df

,required_quantity
Barbecued Chicken,3685.992559
Red Peppers,9239.865447
Green Peppers,5133.676802
Tomatoes,32450.939059
Red Onions,49348.143852
...,...
Luganega Sausage,4349.537067
Onions,2174.768533
Soppressata Salami,5367.962829
Peperoncini verdi,1063.650498


#### create a purchase order

In [207]:
# Create a purchase order DataFrame
purchase_order_df = ingredient_requirements_df.copy()
purchase_order_df = purchase_order_df.rename(columns={'required_quantity': 'required_quantity_in_grams'})
# Add a column for the unit of measure (assuming all ingredients are in grams)
print('PURCHASE ORDER:')
print(purchase_order_df.to_string())

PURCHASE ORDER:
                            required_quantity_in_grams
Barbecued Chicken                          3685.992559
Red Peppers                                9239.865447
Green Peppers                              5133.676802
Tomatoes                                  32450.939059
Red Onions                                49348.143852
Barbecue Sauce                             1228.664186
Bacon                                     17838.982602
Pepperoni                                 20759.519243
Italian Sausage                             329.264634
Chorizo Sausage                            1646.323169
Brie Carre Cheese                           317.423094
Prosciutto                                  317.423094
Pears                                       105.807698
Thyme                                        52.903849
Garlic                                    15909.903744
?duja Salami                               2342.683952
Pancetta                                   3514.0